In [1]:
import osmnx as ox
import matplotlib.pyplot as plt
import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import math

from shapely.geometry import box, Point, LineString
from heapq import heappush, heappop
from itertools import count
from rtree.index import Index

In [3]:
place_name = "Singapore, Central, Singapore"
graph = ox.graph_from_place(place_name)
ox.save_load.save_graphml(graph, filename='Singapore.graphml', folder=None, gephi=False)

# Stores nodes and edges along with other attributes
type(graph)

networkx.classes.multidigraph.MultiDiGraph

In [4]:
nodes, edges = ox.graph_to_gdfs(graph, nodes=True, edges=True)

#to save the dataframe
nodes.to_pickle('data/nodes.pkl')

nodes.head()

,y,x,osmid,highway,ref,geometry
5215092740,1.375299,103.698982,5215092740,NaN,NaN,POINT (103.69898 1.37530)
5215092741,1.376744,103.699105,5215092741,NaN,NaN,POINT (103.69911 1.37674)
5215092742,1.376767,103.698839,5215092742,NaN,NaN,POINT (103.69884 1.37677)
5215092743,1.375342,103.698728,5215092743,NaN,NaN,POINT (103.69873 1.37534)
5215092748,1.376837,103.698116,5215092748,NaN,NaN,POINT (103.69812 1.37684)


In [5]:
def process_edges(edges):
    # Remove list
    i = 0
    
    for x in edges['maxspeed']:
        if type(x) == list:
            edges['maxspeed'][i] = max(x)
        i += 1    

    edges['maxspeed'] = edges['maxspeed'].astype(float)
    

    # Define free-flow time t(u,v) in sec
    travel_time = []
    
    for l,s in zip(edges['length'], edges['maxspeed']):
        if math.isnan(s):
            travel_time.append(float('nan'))
        else:
            t = (3600*l)/(1000*s)
            travel_time.append(t)
            
    edges['travel_time'] = travel_time
    
    
    # Drop unnecessary columns
    edges.drop(['lanes','name','bridge','tunnel','ref','junction','access','width','est_width','area'], axis=1, inplace=True)

In [6]:
process_edges(edges)
edges.head()

/home/orionstar25/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,u,v,key,osmid,highway,service,oneway,length,geometry,maxspeed,travel_time
0,5215092740,5215092743,0,646805965,service,parking_aisle,False,28.577,"LINESTRING (103.69898 1.37530, 103.69873 1.37534)",NaN,NaN
1,5215092740,5215092741,0,"[646806244, 701289234, 701289237, 701289238, 5...",service,parking_aisle,False,207.423,"LINESTRING (103.69898 1.37530, 103.69915 1.375...",NaN,NaN
2,5215092740,5215092741,1,"[701289240, 538846647, 701289239]",service,parking_aisle,False,161.241,"LINESTRING (103.69898 1.37530, 103.69902 1.375...",NaN,NaN
3,5215092741,5215092742,0,646806242,service,NaN,False,29.692,"LINESTRING (103.69911 1.37674, 103.69884 1.37677)",NaN,NaN
4,5215092741,5215092740,0,"[701289240, 701289239, 538846647]",service,parking_aisle,False,161.241,"LINESTRING (103.69911 1.37674, 103.69907 1.376...",NaN,NaN


In [7]:
G = ox.gdfs_to_graph(nodes, edges)
# Define capacity as a random number between 835-905
import random
capacity  = dict()

for i in range(n):
    x = random.randint(835, 905)
    u = edges['u'][i]
    v = edges['v'][i]
    key = edges['key'][i]
    capacity[(u,v,key)] = x

nx.set_edge_attributes(G, capacity, 'capacity')


ox.save_load.save_graphml(G, filename='Singapore_processed.graphml', folder=None, gephi=False)

In [8]:
nodes['highway'].unique()

array([nan, 'crossing', 'turning_circle', 'traffic_signals',
       'motorway_junction', 'turning_loop', 'elevator', 'bus_stop',
       'mini_roundabout', 'stop', 'speed_camera'], dtype=object)

In [9]:
# Filter out all bus stop nodes
# key: node label ,value: location
idx = Index('data/bus_stops')

xx = nodes['x'].astype(float)
yy = nodes['y'].astype(float)
node_id = nodes['osmid']
node_type = nodes['highway']

for node, x, y, kind in zip(node_id, xx, yy, node_type):
    if kind == 'bus_stop':
        idx.insert(int(node), (x,y,x,y))
        
idx.close()